# Prepare supplement data - the ones that needed to use Stata to create 

## Input: "..\0_data\contraceptive_logistics_data_product_site.csv" 

In [1]:
import delimited "..\0_data\supplement_data_raw.csv", clear 
sort site_code product_code year month



. import delimited "..\0_data\supplement_data_raw.csv", 
> clear 
(50 vars, 38,842 obs)

. sort site_code product_code year month


In [2]:
tab year month


           |                                                                month
      year |         1          2          3          4          5          6          7          8          9         10         11         12 |     Total
-----------+------------------------------------------------------------------------------------------------------------------------------------+----------
      2016 |       477        495        613        622        633        665        684        719        744        770        811        798 |     8,031 
      2017 |       833        830        831        831        842        847        881        882        891        891        901        896 |    10,356 
      2018 |       898        896        905        906        912        915        927        949        951        967        965      1,001 |    11,192 
      2019 |     1,014      1,002      1,006      1,068      1,052      1,032      1,035      1,025      1,029          0          0  

## road conditions 

In [3]:
gen poorroads = 0
replace poorroads = 1 if percentregpoor > 0

gen earthroads = 0
replace earthroads = 1  if percentregearth > 0.7

gen asphaltroads = 0
replace asphaltroads  = 1 if precentregasphalt > 0.14



(17,802 real changes made)


(19,836 real changes made)


(19,980 real changes made)


In [4]:
su


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |     38,842    2017.558    1.066351       2016       2019
       month |     38,842     6.31484    3.334871          1         12
      region |          0
    district |          0
 regioncount |     38,842      1.6425    .8138081          1          3
-------------+---------------------------------------------------------
  cidistrict |          0
   site_code |          0
product_code |          0
stock_init~l |     38,842    63.04477     168.974          0       4320
stock_rece~d |     38,842    14.88796    69.96329          0       3534
-------------+---------------------------------------------------------
stock_dist~d |     38,842     14.9388    40.61266          0       1805
stock_adju~t |     38,842    .9978116    39.11994      -1440       3003
   stock_end |     38,842    63.99174     171.053          0       4320
avera

## Time invariant variables: keep "site_code product_code region district" as identifiers for matching, the other variables are informations 

In [5]:
preserve 

keep  site_code product_code  region district product_type   *roads
duplicates drop
su
save "../0_data/time_invariant_supplement.dta", replace 
restore 





Duplicates in terms of all variables

(37,485 observations deleted)


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
      region |          0
    district |          0
   site_code |          0
product_code |          0
product_type |          0
-------------+---------------------------------------------------------
 regionroads |      1,357    99.65291     59.0284         15        206
regionasph~s |      1,357    12.00442    9.979772          0         25
regioneart~s |      1,357    72.07738    54.50041          9        184
regionsurf~s |      1,357    15.57111    10.90589          0         51
regionpoor~s |      1,357    4.871039    6.329126          0         19
-------------+---------------------------------------------------------
   poorroads |      1,357    .4664702    .4990584          0          1
  earthroads |      1,357    .5239499    .4996102          0          1
asph

## Time variant variables - keep "month year site_code product_code region district"   as identifiers for matching, the others variables are informations

### Generate a time index

In [6]:
gen temp_timeindex=ym(year,month)

In [7]:
su


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |     38,842    2017.558    1.066351       2016       2019
       month |     38,842     6.31484    3.334871          1         12
      region |          0
    district |          0
 regioncount |     38,842      1.6425    .8138081          1          3
-------------+---------------------------------------------------------
  cidistrict |          0
   site_code |          0
product_code |          0
stock_init~l |     38,842    63.04477     168.974          0       4320
stock_rece~d |     38,842    14.88796    69.96329          0       3534
-------------+---------------------------------------------------------
stock_dist~d |     38,842     14.9388    40.61266          0       1805
stock_adju~t |     38,842    .9978116    39.11994      -1440       3003
   stock_end |     38,842    63.99174     171.053          0       4320
avera

In [8]:
egen product_site_id=group(site_code product_code)
xtset  product_site_id temp_timeindex



       panel variable:  product_site_id (unbalanced)
        time variable:  temp_timein~x, 672 to 716, but with gaps
                delta:  1 unit


In [9]:
su product_site_id
tsfill 
bysort product_site_id: carryforward site_code product_code region district maxtemp temp pressure relative* rain* visibility windspeed maxsus* storm fog *roads , replace
su product_site_id



    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
product_si~d |     38,842    659.6362    384.9029          1       1357


site_code:  (2,506 real changes made)
product_code:  (2,506 real changes made)
region:  (2,506 real changes made)
district:  (2,506 real changes made)
maxtemp:  (3,444 real changes made)
temp:  (3,444 real changes made)
pressure:  (3,444 real changes made)
relativehumidity:  (3,444 real changes made)
rainfallsnowmelt:  (3,444 real changes made)
rainordrizzle:  (3,444 real changes made)
visibility:  (3,444 real changes made)
windspeed:  (3,444 real changes made)
maxsustainedwindspeed:  (3,444 real changes made)
storm:  (3,444 real changes made)
fog:  (3,444 real changes made)
regionroads:  (2,506 real changes made)
regionasphaltroads:  (2,506 real changes made)
regionearthroads:  (2,506 real changes made)
regionsurfacetreatmentroads:  (2,506 real changes made)
regionpoor

In [10]:
replace year=int(temp_timeindex/12)+1960 if year==.

(2,506 real changes made)


In [11]:
replace month=(1+temp_timeindex-(year-1960)*12) if month==. 

(2,506 real changes made)


## Not sure why it still has gaps after tsfill 

In [12]:
xtset

       panel variable:  product_site_id (unbalanced)
        time variable:  temp_timein~x, 672 to 716
                delta:  1 unit


In [13]:
preserve 

keep temp_timeindex year month site_code product_code region district maxtemp temp pressure relative* rain* visibility windspeed maxsus* storm fog  
su
duplicates drop
su
save "../0_data/time_variant_supplement.dta", replace 
restore 





    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |     41,348    2017.522      1.0662       2016       2019
       month |     41,348    6.305964    3.328858          1         12
      region |          0
    district |          0
   site_code |          0
-------------+---------------------------------------------------------
product_code |          0
        temp |     40,491    27.12557    2.049585          0         31
     maxtemp |     40,491    31.50439    3.182233          0       64.6
    pressure |     40,491    600.3173    497.0187          0     1016.7
relativehu~y |     40,491    78.30246    13.59954          0     129.02
-------------+---------------------------------------------------------
rainfallsn~t |     40,491    94.60818    93.31702          0      870.2
  visibility |     40,491    8.565474    1.449536          0         10
   windspeed |     40,491   